In [1]:
import pandas as pd
import numpy as np
datadir = '../../data/'



In [2]:
# this will be the input from the stock rollover model
ev_df = pd.DataFrame({
    'year': [2022, 2022, 2023, 2023],
    'county': ['King', 'Spokane', 'King', 'Spokane'],
    'nEV': [5, 2, 6, 3],
    'efficiency_kwh_mi': [0.3, 0.3, 0.4, 0.4],
})
ev_df

,year,county,nEV,efficiency_kwh_mi
0,2022,King,5,0.3
1,2022,Spokane,2,0.3
2,2023,King,6,0.4
3,2023,Spokane,3,0.4


AttributeError: 'numpy.ndarray' object has no attribute 'extend'

In [4]:
# Sample dataframe with people and their home county
people_df = pd.DataFrame({
    'person_id': [1]*, [2]*4, 2, 2], 
    'County': ['King']*2 + ['Spokane']*2,
})
# Population dataframe with county, year, and number of people to select
ev_df = pd.DataFrame({
    'Year': [2022, 2022, 2023, 2023],
    'County': ['King', 'Spokane', 'King', 'Spokane'],
    'nEV': [5, 2, 6, 3],
    'efficiency_kwh_mi': [0.3, 0.3, 0.4, 0.4],
})

selected_people = []
for _, row in ev_df.iterrows():
    county = row['County']
    year = row['Year']
    efficiency_kwh_mi = row['efficiency_kwh_mi']
    num_to_select = row['nEV']
    county_df = people_df[(people_df['County'] == county)]
    selected = county_df.sample(n=num_to_select, replace=False, random_state=42)
    # Add the year column to the selected dataframe
    selected['Year'] = year  
    # Add the efficiency column to the selected dataframe
    selected['efficiency_kwh_mi'] = efficiency_kwh_mi  
    selected_people.append(selected)

# Concatenate the selected people into a final dataframe
final_df = pd.concat(selected_people)
final_df  

ValueError: Cannot take a larger sample than population when 'replace=False'

In [2]:
merged_df = pd.read_parquet(datadir+'wa_pop_and_trips_sorted_county.parquet', engine='pyarrow')

In [3]:
merged_df


,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,...,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,destination_county
0,18336634547462657450,10000002909730821040,PRIVATE_AUTO,SOCIAL,HOME,0 days 08:20:42,0 days 08:39:00,2.897196,nan,530250111002,...,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA"
1,1694659094182985471,10000002909730821040,PRIVATE_AUTO,HOME,SOCIAL,0 days 09:12:45,0 days 09:19:56,2.867602,nan,530250111003,...,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,HOME,"Grant County, WA"
2,7721623352602932778,10000002909730821040,ON_DEMAND_AUTO,SHOP,HOME,0 days 09:30:08,0 days 09:49:04,8.929170,nan,530250111002,...,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA"
3,3697919619727259534,10000002909730821040,CARPOOL,SHOP,SHOP,0 days 09:54:21,0 days 09:57:14,0.996870,nan,530250109022,...,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA"
4,9977271661694103189,10000002909730821040,WALKING,SHOP,SHOP,0 days 10:23:28,0 days 10:25:01,0.080298,nan,530250111001,...,english,47.085370,-119.319090,47.132725,-119.213000,NaN,NaN,worked_in_person,PUBLIC,"Grant County, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51727263,17390194002376271909,129762667123447258,PRIVATE_AUTO,SHOP,HOME,0 days 18:08:13,0 days 18:15:40,3.283053,nan,530330303102,...,english,47.291426,-122.343578,47.300378,-122.359055,NaN,NaN,worked_from_home,PUBLIC,"King County, WA"
51727264,884521432970543080,129762667123447258,PRIVATE_AUTO,SHOP,SHOP,0 days 18:22:16,0 days 18:25:16,0.749009,nan,530330303141,...,english,47.291426,-122.343578,47.300378,-122.359055,NaN,NaN,worked_from_home,PUBLIC,"King County, WA"
51727265,7536433276779622397,129762667123447258,PRIVATE_AUTO,HOME,SHOP,0 days 18:48:28,0 days 18:56:31,3.582652,nan,530330300062,...,english,47.291426,-122.343578,47.300378,-122.359055,NaN,NaN,worked_from_home,HOME,"King County, WA"
51727266,3833876188703885727,12976269075468139206,PRIVATE_AUTO,SCHOOL,HOME,0 days 07:44:43,0 days 07:47:12,0.808624,nan,530630024001,...,GQ_language,47.664690,-117.427617,NaN,NaN,47.672023,-117.419789,under_16_or_notinlf,PUBLIC,"Spokane County, WA"


In [13]:
bg_df['destination_bgrp'].value_counts()

destination_bgrp
530530617005    2
530530725054    2
530530635022    2
530530635011    2
530530635012    2
               ..
530239703001    1
530630105013    1
530630105014    1
530630105042    1
530150011006    1
Name: count, Length: 4783, dtype: int64